# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [20]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [22]:
webdriver_path = 'C:\\chromedriver.exe'
browser = webdriver.Chrome(executable_path=webdriver_path)
browser.get(ETtoday_url) # 打開瀏覽器並連到東森新聞雲網頁
driver = browser

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [23]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [24]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 台翰7月營收1.78億創今年來新高　下半年可望維持成長動能

▲台翰精密主營業務。（圖／台翰官網）
記者姚惠茹／台北報導
台翰精密（1336）今（6）日公布7月合併營收1.78億元，月增19.17%，創今年來單月新高，主要是受到疫情趨緩，東南亞各地區政府以漸進式復甦經濟活動，累計今年前7月合併營收9.9億元，年減26.22%。台翰表示，最壞的情況已過，下半年業績可望比上半年好。
台翰表示，上半年營運受到新冠肺炎疫情影響，但菲律賓廠區自5月16日復工後，可見主要客戶的拉貨力道有逐步恢復的跡象，公司與客戶積極拉高產能稼動率以因應整體印表機/事務機的需求，目前產能稼動率已達50%水準，而越南廠的產能利用率也已達70%。
台翰指出，雖然疫情短期影響訂單遞延出貨，但目前已與主要客戶積極針對新機種進行認證，若順利按照時程進度，有機會在今年第3季試量產，未來隨著客戶需求增加，將有助於整體產能規模放大，並強調最壞的情況已過，下半年本業營運有望恢復過往成長表現，可望比上半年好。
台翰說明，目前越南廠區佔整體營收55%、菲律賓廠區約佔35%、東莞廠約佔10%，而且東莞廠將會持續縮編，預計明年僅剩60人左右，作為模具開發設計使用，主要是因為大陸生產成本越來越高，但今年由於有新模具開發，所以第3季的比重則是以東莞廠為主。
根據中國海關總署公布今年上半年最新貿易統計數據顯示，大陸與所有國家及地區的貿易額表現來看，由於大陸企業將生產線轉移至東盟國家，創造大陸與東盟(ASEAN)的進出口貿易額首次升至第一，而且近期歐美、日韓等各國也積極祭出補貼計畫，鼓勵該國企業遷回該國家或東南亞區藉以分散產能布局、確保產品與材料的順利供應，有利於東南亞產業鏈群聚發展。
展望下半年營運，台翰表示，看好東南亞兩個生產基地產能規模放大，貢獻整體營運穩健基礎，雖然東南亞地區政府仍實施防疫措施，但仍以最低限度生產方式發揮最大量產價值，滿足主要客戶的拉貨需求，有助於下半年營運保持良好的成長動能。
台翰強調，公司持續強化模具設計、塑膠射出的量產技術，將憑藉著深耕東南亞市場超過15年，積極爭取更多與不同產業客戶合作的可能性，藉以保持高度的訂單能見度，並對整體營運帶來正面的挹注。

圖文／鏡週刊
今年33歲的韋禮安，睽違4年推出第5張專輯《Sounds of My Life》，紀錄生活中所在乎的聲音，是一張充滿愛與感謝的作品。創作音樂感性十足的他，3年前就開始為退休鋪路，透過大量閱讀奠定長期投資的道路，理性分析ETF最適合自己。
從出道專輯裡的〈慢慢等〉、〈有沒有〉，到電視劇片頭曲〈還是會〉、〈女孩〉，韋禮安的作品傳唱度極高，深受大眾歡迎。但好好的歌手怎麼會開始投資呢？原來是出於對未來的不安感，「2017年快邁入30歲時，我意識到演藝人員工作不穩定，退休金得做好規劃。」韋禮安說。
「扣掉每月生活費，幾乎80%收入都放在經營公司與ETF投資上。」他坦言，當歌手又開公司，職涯上已承受相當高的風險，因此投資上應選擇相對穩健的ETF。
其實早在13年前，剛上大學的韋禮安就在父親的規劃下，開始定期定額買基金，直到現在都還持續扣款。出身書香世家的他，父母皆是外文系教授，「家人在投資理財上很保守，當初也是先跟爸爸討論，才慢慢開始投資海外ETF。」韋禮安解釋。
「投資多年的基金累積報酬雖可觀，但換算成年報酬也只比定存好一點，反而ETF績效更好。」韋禮安表示，目前投資部位有4分之3放在ETF上，分別是：VTI（範圍涵蓋美股大中小型公司，等於買下美國整體市場表現）、VEU（投資美國以外的已開發國家與新興市場，共持有45個國家）、BND（主要投資美國公債、投資級公司債，以及美元計價的國際債券等）。
「VTI加VEU的組合，等於投資全世界股市，BND則是美國整體債券市場。」韋禮安分析。而現年33歲他，股債比約7比3，投資3年下來平均年賺約7%。
事實上，韋禮安買的3檔ETF都是美國先鋒集團（Vanguard）發行的產品，為何獨鍾此集團？韋禮安解釋，先鋒的創辦人約翰‧伯格（John Bogle）是指數基金教父，集團長年深耕ETF商品，且內扣費用率最低。以他投資的ETF為例，VTI總管理費用0.03%、VEU為0.08%、BND為0.035%，「比起共同基金動不動就3%的內扣費用，投資成本差了100倍。」。
----------------------------------------------------------------------

[14] 張張拿1700元！兆豐金「填息不用一年」心動了　網看傻：誰要定存

▲原PO好奇，有閒錢卻不買股票是不是很傻？（圖

獲利是推升股價的原動力，台股在這波疫情影響之下，突破30年前高點，攻抵13000點，幕後仰賴的就是企業本身的獲利成長動能，不少上市櫃企業上半年營運絲毫無畏疫情干擾，依然繳出亮麗成績單，成為支撐股價之後盾。分析財報除了看獲利、EPS之外，毛利率與營益率的變化也是法人評判一家公司本業經營績效之重要依據。在半年報公布前夕，一起來搶先布局財報黑馬股。
文／馮欣仁
這波台股創下歷史新高點，但細看類股走勢卻呈現兩極化，電子股明顯強過傳產股。雖然全球疫情肆虐，但科技創新與疫情帶動居家辦公之商機，促使晶圓代工、ＩＣ設計、封測等半導體產業上半年營運表現出色，半年報也繳出不錯的成績單。反觀，塑化、鋼鐵、紡織、汽車零組件等傳產族群，受到油價下跌與封城關係，營運明顯受到衝擊，因此，上半年營收與獲利普遍均較去年同期衰退，同時拖累股價表現。
八月十四日之前，所有上市櫃企業將公布第二季財報，屆時上半年獲利表現就能一清二楚。本期封面故事即從財報出發，分別從不同角度去分析，除了看ＥＰＳ之外，毛利率與營益率的變化，也是攸關廠商本業獲利與競爭力強弱的重要指標。就目前已公告的企業獲利來看，ＩＣ設計族群普遍表現不錯。
ＩＣ設計獲利強悍 義隆拚賺一股本
例如：ＩＣ設計龍頭聯發科受惠於５Ｇ手機晶片出貨成長，以及智慧家庭包括WiFi、PMIC、ASICs等產品復甦，帶動第二季毛利率衝上四三．五％，創近四年半以來新高，ＥＰＳ達四．五八元。展望第三季，公司樂觀看待５Ｇ手機晶片出貨持續成長，營收可望較上季成長二二至三○％、毛利率也有望向上提升。法人看好，聯發科在５Ｇ晶片市占率攀高，紛紛上修目標價，最高上看一二○○元。
除了聯發科之外，面板驅動ＩＣ聯詠、影音晶片瑞昱、高速傳輸晶片譜瑞ＫＹ第二季財報也相當亮眼。其中，譜瑞第二季更大賺一個股本，累計上半年ＥＰＳ已達十八．三三元。法人看好譜瑞明年在Mini LED規格導入iPad及Macbook拉開升級序幕，加上Mini LED滲透率攀升，帶動高毛利率的時脈控制晶片（eDP TCON）平均單價逐年向上，近來也上修目標價至一五○○元，直追矽力ＫＹ與祥碩。
全球觸控ＩＣ龍頭義隆受惠疫情帶動ＮＢ需求強勁，高毛利產品出貨比重增加，促使第二季稅後淨利達九．一八億元，季增一七○○％，年增一一七％，ＥＰＳ達三．一五元，累計上半年ＥＰＳ達三．三三元。展望下半年，公司持樂觀看待，七月營收更衝上十

▲只要能有效控管自己的花費，不記帳也無妨。（圖／取自Pixabay）
記者曾筠淇／綜合報導
不少人會透過記帳來控管自己的資金，不過如果能有效掌控預算、花費，就算不記帳也無所謂。斜槓護理師Elaine理白小姐先前表示，「我已經兩個月沒記帳了」，因為買的都是必需品，且能確實受到預算的限制，因此就能更舒服自在的生活，不被形式綁架。
文中指出，記帳的目的其實是為了掌握金錢流向，並找出不必要的消費，且記帳主要著重在事後檢討，預算則是事前規畫。多數人在消費時，都會認為那樣東西是必要的花費，但有了預算的限制後，才會真正去思考自己是否有需要購買，進而在反思後，確實做到「只購買需要的東西」這件事。
Elaine理白小姐強調，她並不是要鼓勵大家不記帳，而是希望每個人都能選擇自己最舒服的方式來理財，不必被形式綁架。
----------------------------------------------------------------------

[28] 財訊／龍巖李世聰親訪　跨界吃下錠嵂3成持股的盤算

文／財訊新聞中心
7月22日，龍巖宣布砸下4.2億元台幣買下錠嵂逾3成持股，一週後李世聰親自帶隊拜訪錠嵂創辦人毛以孝，會談兩個小時，雙方究竟各有什麼盤算？
7月最後一週，台股在台積電大漲帶動下再創歷史新高，而持有台積電的投資人也紛紛被起底，其中最引發市場熱議的「新科股神」莫過於龍巖。根據公開資訊，龍巖董事會於今年3月授權董事長買股投資以提升資金運用效能後，陸續買入台積電、大立光、波克夏和谷歌母公司Alphabet股票，短短4個月賺進2.15億元，帳面整體獲利17%。
龍巖集團總裁李世聰除了在股市展現「神操作」，也將重心轉向重新布局事業版圖，而他鎖定的對象就是近兩年崛起的保經代公司。7月22日，龍巖宣布砸下4.2億元台幣，向錠嵂部分股東所集結成立的京倫設計工程公司，買下錠嵂逾3成持股。
即使李世聰對外都宣稱，已不過問公司事情，但據透露，7月底李世聰是親自率隊拜訪錠嵂創辦人毛以孝。雙方雖然各為一方之霸，但過去並不相識，對於剛出清彰銀持股的李世聰，這次一口氣吃下錠嵂3成持股，業界議論紛紛。
龍巖參股錠嵂保經公司股票，主要是看好錠嵂是國內前3大保經之一，營收及獲利穩定、財務也十分健全，是很好的財務投資。不過據說初次見面，就閉門談了將近兩個小時，可見龍巖也想透過保經代通路加速推廣生前

▲智伸科。（圖／翻攝自智伸科官網）
記者楊絡懸／台北報導
汽車動力暨安全零組件大廠「智伸科」（4551）公布2020年7月合併營收為6.67億元，月增21.34%，年成長4.34%，是今年疫情後首次轉為正成長的月份；累計前7月營收為36.95億元，年減14.66%。
智伸科表示，7月營收受惠於主要汽車客戶針對旗下汽油高壓泵浦本體、缸內直噴引擎（Gasoline Direst injection, GDI）等相關汽車零組件產品的拉貨力道逐步增溫，醫療及電子兩大事業部的出貨表現也分別較上月成長，減緩全球車市受疫情的衰退衝擊，目前台灣、中國共3個生產基地的平均產能利用率已從疫情發生後持續增加。
根據標準普爾全球市場（S&P Global Market Intelligence）研究指出，雖今年度受大環境不確定因素影響下，今年度全球電動車銷量將年減 5.8%，但隨著下半年新車銷售市場持續復甦，預估至2024年全球電動車的銷量將達620萬輛，相較於2019年銷量成長幅度達3倍。
智伸科認為，旗下馬達、轉向、安全系統等相關零組件產品為電動車車款的關鍵零組件之一，已積極與主要客戶開發系統中相關零組件的新產品，有望隨著產品的滲透率提升，將推進集團搶占新能源車與電動車市場份額速度優於同業之表現，保持未來營運長期穩健成長。
展望2020年第三季，智伸科維持審慎樂觀看法。智伸科表示，即便全球景氣仍存在著不確定性因子的風險， 但觀察各國新車銷售市場年減率的衰退幅度有接續收窄的情況，將對汽車產業帶來正面的助益。
智伸科卯足全力與主要客戶密切討論後續訂單出貨時程的安排，同時也不間斷的與客戶鑽研開發新技術應用的新零組件產品，以期客戶的拉貨力道提升。
此外，智伸科指出，伴隨著公司優化產品結構、嚴控費用支出、彈性調配產能等多方面執行的效益展開，貢獻集團今年的整體營運有機會繳出下半年優於上半年的亮麗表現。
----------------------------------------------------------------------

[40] 大同市場派股臨會過關與否　經長王美花：下周可望定案

▲經濟部長王美花。（圖／記者湯興漢攝）
記者林淑慧／台北報導
大同經營權之爭持續延燒，市場派二方股東先後申請召集股東臨時會，經濟部長王美花今（6）日表示，大同公司針對二案均已回文，目前金管會已在

▲元大金控總經理翁健（左）。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
台股今年3月大跌後已重返「萬二」，甚至一度創下歷史新高，吸引不少年輕人進場。元大金控總經理翁健今（6）日表示，台股目前表現不錯，看起來是「有基之彈」，也帶動年輕人進場意願；根據統計，今年以來元大證券線上開戶客戶近5成為30歲以下年輕人。
元大金控2020年上半年稅後盈餘110.63億元，每股盈餘達0.91元，元大金控總經理翁健表示，整體來看，獲利儘管稍受疫情影響，但與原先公司預估目標差不多，目前股市狀況不錯，看起來是「有基之彈」，下半年應是「審慎樂觀」。
翁健進一步指出，現在台股表現非常穩定，看起來是「有基之彈」，加上近期新台幣升值、資金回流、外資匯入等，台股殖利率在國際市場也數一數二，因此受到投資人青睞。
至於年輕人進場比例攀高，翁健認為，整體環境、景氣等影響，部分產業表現還不錯、台股殖利率又高，加上投資理財觀念越來越能獲得大家認同。
根據統計，元大證券今年1至6月新增總開戶數11萬12戶，其中6月新增1.8萬戶，比5月增加3,000人；今年因為疫情影響，增加線上開戶需求，線上開戶客戶近5成為30歲以下年輕人。整體來看，元大證券目前總開戶數超過400萬戶。
翁健也提及，近期集團準備推動定期定額，主要是他認為「任何時候買都來得及」，這是很重要的觀念，股價上漲時，買的單位數較少，反之則變多，因此鼓勵投資人可善用定期定額來投資。
翁健說，「定期定額是一個好的投資方向」，而近期高股息優質龍頭基金淨值11.9元創新高，也適合推定期定額，目前仍在熱身中，預計8月起正式開跑。
----------------------------------------------------------------------

[51] 買保險別被國稅局盯上！帶重病、高齡投保等6大NG動作　易被抓規避遺產稅

▲長輩過世前短期內大量投保，也會被國稅局盯上 。（圖／取自免費圖庫Pixabay）
記者吳靜君／台北報導
許多人都會透過保單的規劃，來作為遺產稅的節稅工具，不過如果利用保險刻意規避遺產稅，小心被國稅局盯上！
根據財政部的「實務上死亡人壽保險金依實質課稅原則核課遺產稅案例」顯示，有被繼承人在2009年過世，但是在2004年的4月、7月與9月買了50張保單，其中躉繳的投資型保單2張以及養老保險48張，共計1800萬

▲專家預估下周國內油價將調漲。（圖／記者姜國輝攝）
記者林淑慧／台北報導
用車族加油要快！隨著美國商用原油庫存下降，國際油價持續波動，在考量亞鄰競爭國最低價的條件下，專家預估下周國內油價將調漲，汽柴油每公升零售價約各調漲0.1元，此為國內油價連續三周呈現上漲走勢；若調幅如預期，九五汽油每公升將突破24元大關。
依據浮動油價調整機制作業原則，在考量亞鄰競爭國最低價的條件下，調整後的參考零售價格分別為：九二無鉛汽油為22.5元、九五無鉛汽油24元、九八無鉛26元、超級柴油19.8元。
油價專家指出，美國商用原油庫存下降，加上美國製造業採購經理人表現強勁，影響國際油價持續波動；累算至8月5日之7D3B週均價為 43.56 美元，較上週43.06上漲0.50美元。新台幣兌美元匯率29.525較上週29.5071貶值0.018。
依浮動油價調整機制作業原則，以及不考量亞鄰競爭國最低價及平穩措施的條件下，預估下週汽、柴油零售價格每公升約各調漲0.5元及0.7元，台灣中油可望吸收部分漲幅，實際調整金額請於週日中午12時詳見台灣中油全球資訊網暨經濟部網站。
 
----------------------------------------------------------------------

[60] 生華科新藥與全美最大醫療體系Banner Health合作　進行新冠人體臨床

▲左起為生華科董事長胡定吾、總經理宋台生。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
生華科（6492）今（6）日宣布，新藥CK2抑制劑Silmitasertib (CX-4945)，攜手全美最大的醫療體系的Banner Health簽訂合作備忘錄，直接運用新藥Silmitasertib在新冠肺炎患者治療上，進行人體臨床試驗，成為台灣研發新藥最早用於治療新冠肺炎患者的生技公司。
生華科表示，目前將雙軌並進，Banner Health除了將盡速向美國FDA申請擴大取得人體臨床試驗(Expanded Access IND)，為個別單一患者進行治療，同時也將提出由研究者發起主導的IIT臨床試驗(Investigator-Initiated Trial, IIT)，以快速取得較具規模的人體臨床數據。
生華科指出，若對新冠患者具正向療效，生華科將和美國FDA討論爭取緊急使用授權EUA，並進一步討論樞紐性試

AttributeError: 'NoneType' object has no attribute 'find_all'